In [1]:
import pandas as pd
import numpy as np
import random
import json
import time
import datetime
import os

from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig

import torch
from transformers import BertTokenizer
from transformers import get_linear_schedule_with_warmup


In [2]:
train_fname = '../inputs/dataset/smart2021-AT_Answer_Type_Prediction/dbpedia/task1_dbpedia_train.json'
# train_fname = '../inputs/dataset/smart2021-AT_Answer_Type_Prediction/dbpedia/2021_dbpedia_0.9train.json'
dbpedia_train_df = pd.read_json(train_fname)

# # cleaning DBpedia dataset
# print(len(dbpedia_train_df))
# dbpedia_train_df = dbpedia_train_df[dbpedia_train_df.question.notna()]
# print(len(dbpedia_train_df))
# dbpedia_train_df= dbpedia_train_df[dbpedia_train_df.category.notna()]
# print(len(dbpedia_train_df))
# dbpedia_train_df= dbpedia_train_df[dbpedia_train_df.type.notna()]
# print(len(dbpedia_train_df))
# dbpedia_train_df.head()

In [3]:
dbpedia_train_df['category'].unique()

array(['boolean', 'resource', 'literal'], dtype=object)

In [4]:
dbpedia_train_df['category'].value_counts()

resource    30226
literal      4217
boolean      2227
Name: category, dtype: int64

In [5]:
for idx in dbpedia_train_df.index:
    if dbpedia_train_df['category'][idx] == 'literal':
        dbpedia_train_df['category'][idx] = ''.join(dbpedia_train_df['type'][idx])
classes = dbpedia_train_df['category'].value_counts()
print(classes)
dbpedia_train_df.head()

<ipython-input-5-c8de9a6a5e6e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dbpedia_train_df['category'][idx] = ''.join(dbpedia_train_df['type'][idx])


resource    30226
boolean      2227
string       1655
number       1314
date         1248
Name: category, dtype: int64


,id,question,category,type
0,0,Was Jacqueline Kennedy Onassis a follower of M...,boolean,[boolean]
1,1,What is the name of the opera based on Twelfth...,resource,"[dbo:Opera, dbo:MusicalWork, dbo:Work]"
2,2,When did Lena Horne receive the Grammy Award f...,date,[date]
3,3,Do Prince Harry and Prince William have the sa...,boolean,[boolean]
4,5,Which is the hierarchical BrainInfo ID of the ...,string,[string]


In [6]:
label_map_dic = {
      'resource': 0,
      'boolean': 1,
      'string': 2,
      'number': 3,
      'date': 4
      }
dbpedia_train_df['5classes'] = dbpedia_train_df['category'].map(label_map_dic)
dbpedia_train_df.head()

,id,question,category,type,5classes
0,0,Was Jacqueline Kennedy Onassis a follower of M...,boolean,[boolean],1
1,1,What is the name of the opera based on Twelfth...,resource,"[dbo:Opera, dbo:MusicalWork, dbo:Work]",0
2,2,When did Lena Horne receive the Grammy Award f...,date,[date],4
3,3,Do Prince Harry and Prince William have the sa...,boolean,[boolean],1
4,5,Which is the hierarchical BrainInfo ID of the ...,string,[string],2


In [7]:
dbpedia_date_df = dbpedia_train_df[dbpedia_train_df['5classes']==4]

In [8]:
dbpedia_date_df.sample(5)

,id,question,category,type,5classes
5331,6710,When was Yukio Mishima nominated for a Nobel P...,date,[date],4
5472,6890,When did William McPherson Allen step down as ...,date,[date],4
7994,10033,When was the first performance of La Traviata ...,date,[date],4
10395,13021,When was Robert Frank awarded the Hasselblad A...,date,[date],4
8741,10968,when did lyudmila zubovich tenure as the head ...,date,[date],4


In [9]:
# Instantiate the Bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [10]:
input_ids = []
attention_masks = []
labels = []

MAX_LENGTH = 36

# For every sentence...
for i,row in dbpedia_train_df.iterrows():
    # sent = str('[CLS]') + row['question'] + str('[SEP]')
    sent = row['question']
    labels.append(int(row['5classes']))
    encoded_dict = tokenizer.encode_plus(
                      sent,                          # Sentence to encode.
                      add_special_tokens = True,     # Add '[CLS]' and '[SEP]'
                      max_length = MAX_LENGTH,       # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      return_attention_mask = True,  # Construct attn. masks.
                      return_tensors = 'pt',         # Return pytorch tensors.
                  )
  
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
  
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
print(labels.shape)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2198: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


torch.Size([36670])


In [11]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

33,003 training samples
3,667 validation samples


In [12]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [13]:
model = BertForSequenceClassification.from_pretrained(
               'bert-base-uncased', 
               num_labels = len(label_map_dic), # The number of output labels. 2 for binary classification.  
               output_attentions = False,
               output_hidden_states = False)   

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# send model to device
model.to(device) 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [14]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 2,I have already seen that the model starts overfitting beyound 2 epochs
epochs = 5

# Total number of training steps is [number of batches] x [number of epochs]. 
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                       num_warmup_steps = 0, # Default value in run_glue.py
                       num_training_steps = total_steps)

In [15]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [16]:
# Set the seed value all over the place to make this reproducible.
seed_val = 66

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# store a number of quantities such as training and validation loss,validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # training
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader andcopy each tensor to the GPU
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing abackward pass.
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids, 
                token_type_ids=None, 
                attention_mask=b_input_mask, 
                labels=b_labels)

        # Accumulate the training loss over all of the batches
        loss= outputs[0]
        logits = outputs[1]  
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    
    # ========================================
    # Save model of every epoch
    output_dir = '../outputs/models_dbpedia_final/'
    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    models_dir = output_dir + 'BERT_model_5classes_epoch' + str(epoch_i + 1)
    # Create output directory if needed
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)

    print("Saving model to %s" % models_dir)

    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(models_dir)
    tokenizer.save_pretrained(models_dir)
    
    # ========================================
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0.0
    total_eval_loss = 0.0
    nb_eval_steps = 0.0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        
            outputs= model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
            
        # Accumulate the validation loss.
        loss= outputs[0]
        logits = outputs[1] 
        
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.4f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

======== Epoch 1 / 5 ========
Training...
  Batch    40  of  1,032.    Elapsed: 0:00:04.
  Batch    80  of  1,032.    Elapsed: 0:00:08.
  Batch   120  of  1,032.    Elapsed: 0:00:11.
  Batch   160  of  1,032.    Elapsed: 0:00:15.
  Batch   200  of  1,032.    Elapsed: 0:00:19.
  Batch   240  of  1,032.    Elapsed: 0:00:23.
  Batch   280  of  1,032.    Elapsed: 0:00:26.
  Batch   320  of  1,032.    Elapsed: 0:00:30.
  Batch   360  of  1,032.    Elapsed: 0:00:33.
  Batch   400  of  1,032.    Elapsed: 0:00:37.
  Batch   440  of  1,032.    Elapsed: 0:00:40.
  Batch   480  of  1,032.    Elapsed: 0:00:44.
  Batch   520  of  1,032.    Elapsed: 0:00:48.
  Batch   560  of  1,032.    Elapsed: 0:00:51.
  Batch   600  of  1,032.    Elapsed: 0:00:56.
  Batch   640  of  1,032.    Elapsed: 0:01:00.
  Batch   680  of  1,032.    Elapsed: 0:01:04.
  Batch   720  of  1,032.    Elapsed: 0:01:09.
  Batch   760  of  1,032.    Elapsed: 0:01:13.
  Batch   800  of  1,032.    Elapsed: 0:01:17.
  Batch   840  of 

In [17]:
import os
# Display floats with two decimal places.
pd.set_option('precision', 4)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# output_dir = '../outputs/models_dbpedia/'
# # Create output directory if needed
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

df_stats.to_csv(output_dir + 'df_category_stats.csv',index=None)

df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.1323,0.0707,0.9826,0:01:43,0:00:03
2,0.0595,0.0681,0.9840,0:01:52,0:00:03
3,0.0281,0.0795,0.9842,0:01:51,0:00:03
4,0.0113,0.0913,0.9851,0:01:51,0:00:03
5,0.0028,0.1044,0.9864,0:01:51,0:00:03


In [18]:
# models_dir = output_dir + 'BERT_model_5classes' + epoch

# # Create output directory if needed
# if not os.path.exists(models_dir):
#     os.makedirs(models_dir)

# print("Saving model to %s" % models_dir)

# # Save a trained model, configuration and tokenizer using `save_pretrained()`.
# # They can then be reloaded using `from_pretrained()`
# model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
# model_to_save.save_pretrained(models_dir)
# tokenizer.save_pretrained(models_dir)